In [1]:
library(terra)
library(tidyterra)
library(dplyr)
library(randomForest)



terra 1.7.71


Attaching package: ‘tidyterra’


The following object is masked from ‘package:stats’:

    filter



Attaching package: ‘dplyr’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine




In [2]:
wa_shp <- vect("data/wa_shp_diss.gpkg")
wa_ecoreg <- vect("data/wa_l3_eco/wa_eco_l3.shp") |> project("EPSG:3740") |> tidyterra::select(US_L3NAME)
wa_ecoreg

 class       : SpatVector 
 geometry    : polygons 
 dimensions  : 9, 1  (geometries, attributes)
 extent      : 371124.2, 971138, 5043576, 5444546  (xmin, xmax, ymin, ymax)
 coord. ref. : NAD83(HARN) / UTM zone 10N (EPSG:3740) 
 names       :        US_L3NAME
 type        :            <chr>
 values      :      Coast Range
               Columbia Plateau
                 Blue Mountains

In [3]:
ppt <- rast("data/stack_data/PRISM/wa_ppt.tif")
tmean <- rast("data/stack_data/PRISM/wa_tmean.tif")
vpd <- rast("data/stack_data/PRISM/wa_vpd.tif")
clim_stack <- c(ppt, tmean, vpd)

In [7]:
list_files <- list.files(path = "data/stack_data", full.names = TRUE, pattern = ".tif", include.dirs = FALSE)
list_files
terr_stack<- rast(list_files)
terr_stack <- terra::subset(terr_stack, "wa_dinfpoint", negate = TRUE)
terr_stack

[1] "data/stack_data/wa_dem_diss_prj3740.tif"
 [2] "data/stack_data/wa_dev27msk.tif"        
 [3] "data/stack_data/wa_dev3msk.tif"         
 [4] "data/stack_data/wa_dev81msk.tif"        
 [5] "data/stack_data/wa_dinfpoint.tif"       
 [6] "data/stack_data/wa_dinfsca.tif"         
 [7] "data/stack_data/wa_planc.tif"           
 [8] "data/stack_data/wa_profc.tif"           
 [9] "data/stack_data/wa_rugged.tif"          
[10] "data/stack_data/wa_slope.tif"           
[11] "data/stack_data/wa_twi.tif"

class       : SpatRaster 
dimensions  : 16959, 25378, 10  (nrow, ncol, nlyr)
resolution  : 23.64304, 23.64304  (x, y)
extent      : 371131.6, 971144.8, 5043580, 5444542  (xmin, xmax, ymin, ymax)
coord. ref. : NAD83(HARN) / UTM zone 10N (EPSG:3740) 
sources     : wa_dem_diss_prj3740.tif  
              wa_dev27msk.tif  
              wa_dev3msk.tif  
              ... and 7 more source(s)
names       : NASAD~9w125,  wa_dev27,   wa_dev3,  wa_dev81, wa_dinfsca, wa_planc, ... 
min values  :   -35.65501, -13.49994, -13.55343, -18.42358,         ? ,       ? , ... 
max values  :  4373.03906,  10.51795,  13.55344,  13.27825,         ? ,       ? , ... 

In [8]:
wa_spec<- rast("data/Spectral/WA_median_mosaic.tif")
wa_spec <- terra::subset(wa_spec, "uncertainty", negate = TRUE)
wa_spec

class       : SpatRaster 
dimensions  : 13366, 20286, 6  (nrow, ncol, nlyr)
resolution  : 30, 30  (x, y)
extent      : 362550, 971130, 5043570, 5444550  (xmin, xmax, ymin, ymax)
coord. ref. : NAD83(HARN) / UTM zone 10N (EPSG:3740) 
source      : WA_median_mosaic.tif 
names       : tree_~cover, NDVI_median, MNDWI_median, EVI_median, SAVI_median, NDYI_median 
min values  :           0,  -0.9962001,   -0.9924589,  -4.290172,  -0.3153680,   -2.461545 
max values  :          86,   0.9955124,    0.9966356,   7.263647,   0.8293253,    2.052344 

In [9]:
wa_pts <- vect("data/wa_all_pts.gpkg")
wa_pts

 class       : SpatVector 
 geometry    : points 
 dimensions  : 46607, 1  (geometries, attributes)
 extent      : 372213.1, 974284.3, 5040111, 5444105  (xmin, xmax, ymin, ymax)
 source      : wa_all_pts.gpkg
 coord. ref. : NAD83(HARN) / UTM zone 10N (EPSG:3740) 
 names       : class
 type        : <chr>
 values      :   WET
                 WET
                 WET

In [10]:
datapath <- paste0(getwd(), "/data/regional/")
datapath

[1] "/home/ajs0428/WIP Testing/data/regional/"

In [18]:
eco_crop_func <- function(stack, ecoreg, path) {
    for(i in 1:length(ecoreg)){
       terra::mask(stack, ecoreg[i],
                    filename = paste0(path,deparse(substitute(stack)), gsub(" ", "", paste0(values(ecoreg[i]))), ".tif"), 
                   overwrite = TRUE,
                  steps = nrow(stack[[1]]))
    }
}


In [19]:
terraOptions(memmin = 0.1, memfrac = 0.7, memmax = 0.1)

In [20]:
terra::mem_info(wa_spec)


------------------------
Memory (GB) 
------------------------
check threshold : 0.1 (memmin)
available       : 0.1 (memmax)
allowed (70%)   : 0.07
needed (n=1)    : 12.12
------------------------
proc in memory  : FALSE
nr chunks       : 248
------------------------


In [ ]:
eco_crop_func(terr_stack, wa_ecoreg, datapath)